## To merge:
 - Mathew's matlab data
 - my cluster identification data
 - mgefinder
 - breseq (one day, hopefully)


#### Mathew's data
eventually transformed parts of data into csv

In [12]:
import os
import pandas as pd
# Matthews_dir=os.getcwd()+"/PycharmProjects/mge-project/data/Mathews_data/"
# # get the filename list
# filenames=[f for f in os.listdir(Matthews_dir) if f.endswith('.csv')]
# #example how to get the tabl
# ATCU=pd.read_csv(Matthews_dir+filenames[1])
# print(filenames)

In [19]:
# add same strain data to awkward array so that each isolate has 
#   1. whether it is a part of a same-strain subset (true/false for filtering)
#   2. the # of SNPs between it and its colleagues
import sys
sys.path.append('PycharmProjects/mge-project/scripts/')
from presnakemake_processing import process_same_strain_file
same_strain_list=process_same_strain_file("~/PycharmProjects/mge-project/workflow/config/All_same_strain_pairs.xlsx")

print(same_strain_list)

      RandomID Seqplates
0        20643      6_F7
1        20643     6_G11
2        23595      3_H7
3        26244     24_A8
4        26666      5_D3
...        ...       ...
1195   3238816     23_A8
1196   3242814    23_H11
1197   3248787     12_A4
1198   3251316      5_C1
1199   3263632     15_H8

[1200 rows x 2 columns]


#### My cluster data
Plan data structure:

Person

1. RandomID
2. NewRandomID
3. Same-strain case
   1. CaseNum
   2. Isolate
      1. GFF data
      2. RowNum
      3. IsoNum
      4. SampleDate
      5. RawSeqsDir
      6. RawSubDirName
      7. SeqsDir
      8. SeqsSubDirName
      9. ReadLength
      10. Used
      11. Used_old
      12. LowQualityThrs
      13. Freezer_pos
   2. Pangenome
      1. gene names
      2. gene IDs
      3. coverage data
      4. possible other roary input
   3. MGEfinder
   4. Breseq
   5. Identified MGEs
4. Antibiotics purchase
...


## Task #1: import all data into person
### Experimenting with Awkward Array

In [20]:
import awkward as ak
import pprint
import pandas as pd
import numpy as np
import json

isolates_list=pd.read_excel("~/PycharmProjects/mge-project/workflow/config/isolates_list_Maccabi_E.coli_UTI_SeqPlates1to25.xlsx",
        engine="openpyxl")
isolates_list=isolates_list \
    .dropna(axis=1,how="all") \
    .dropna(axis=0,how="all") \
    .convert_dtypes(convert_floating=False)
isolates_list.SampleDate=isolates_list.SampleDate.apply(str)
display(isolates_list)

,RowNum,IsoNum,CaseNum,SiteString,SampleDate,RawSeqsDir,RawSubDirName,SeqsDir,SeqSubDirName,ReadLength,Used,Used_old,LowQualityThrs,Freezer_pos,NewRandomID,RandomID
0,1,40258178.0,150429,Urine,2017-12-10,/media/kishonylab/KishonyStorage/Illumina_raw/...,Sample_Maccabi_Ecoli_SeqPlate1_A1,/media/kishonylab/KishonyStorage/Sequencing_an...,Sample_Maccabi_Ecoli_SeqPlate1_A1,125,1,1,2,1_E2,171794,171794
1,2,40251080.0,150429,Urine,2017-12-07,/media/kishonylab/KishonyStorage/Illumina_raw/...,Sample_Maccabi_Ecoli_SeqPlate1_B1,/media/kishonylab/KishonyStorage/Sequencing_an...,Sample_Maccabi_Ecoli_SeqPlate1_B1,125,1,1,2,1_G2,171794,171794
2,3,40280202.0,2475530,Urine,2017-12-11,/media/kishonylab/KishonyStorage/Illumina_raw/...,Sample_Maccabi_Ecoli_SeqPlate1_C1,/media/kishonylab/KishonyStorage/Sequencing_an...,Sample_Maccabi_Ecoli_SeqPlate1_C1,125,1,1,2,1_B6,2811778,2811778
3,4,40229887.0,2475530,Urine,2017-12-06,/media/kishonylab/KishonyStorage/Illumina_raw/...,Sample_Maccabi_Ecoli_SeqPlate1_D1,/media/kishonylab/KishonyStorage/Sequencing_an...,Sample_Maccabi_Ecoli_SeqPlate1_D1,125,1,1,2,1_C6,2811778,2811778
4,5,40230930.0,2782671,Urine,2017-12-06,/media/kishonylab/KishonyStorage/Illumina_raw/...,Sample_Maccabi_Ecoli_SeqPlate1_E1,/media/kishonylab/KishonyStorage/Sequencing_an...,Sample_Maccabi_Ecoli_SeqPlate1_E1,125,1,1,2,1_A8,2950038,2950038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1735,2008,41072758.0,3142088,Urine,2018-02-20 00:00:00,/media/kishonylab/KishonyStorage/Illumina_raw/...,Sample_Maccabi_Ecoli_SeqPlate17_H4,/media/kishonylab/KishonyStorage/Sequencing_an...,Sample_Maccabi_Ecoli_SeqPlate17_H4,125,1,1,2,'22_D1',3125091,<NA>
1736,2009,41079010.0,2512442,Urine,2018-02-21 00:00:00,/media/kishonylab/KishonyStorage/Illumina_raw/...,Sample_Maccabi_Ecoli_SeqPlate17_H5,/media/kishonylab/KishonyStorage/Sequencing_an...,Sample_Maccabi_Ecoli_SeqPlate17_H5,125,1,1,2,'22_D3',1679582,<NA>
1737,2010,41103812.0,1843996,Urine,2018-02-22 00:00:00,/media/kishonylab/KishonyStorage/Illumina_raw/...,Sample_Maccabi_Ecoli_SeqPlate17_H7,/media/kishonylab/KishonyStorage/Sequencing_an...,Sample_Maccabi_Ecoli_SeqPlate17_H7,125,1,1,2,'22_H5',2395687,<NA>
1738,2011,41095542.0,538390,Urine,2018-02-22 00:00:00,/media/kishonylab/KishonyStorage/Illumina_raw/...,Sample_Maccabi_Ecoli_SeqPlate17_H8,/media/kishonylab/KishonyStorage/Sequencing_an...,Sample_Maccabi_Ecoli_SeqPlate17_H8,125,1,1,2,'22_A7',606367,<NA>


In [21]:

#created nested struct and awkward array from excel
df=isolates_list
ilj = (df.groupby(['NewRandomID','RandomID','CaseNum'])
             .apply(lambda x: x[['IsoNum', 'RowNum','SampleDate','SeqSubDirName','LowQualityThrs', 'Freezer_pos', 'Used', 'SiteString']]
             .to_dict("records"))
             .reset_index()
             .rename(columns={0:'Isolates'})
             .to_json(orient='records'))
#print(json.dumps(json.loads(ilj), indent=2, sort_keys=True))

ila=ak.from_json(ilj)

#add raw seqname to struct
counts=ak.num(ila.Isolates)
Seqname=[str(s).replace('Maccabi_Ecoli_SeqPlate','').replace('Sample_','') for s in ak.flatten(ila.Isolates.SeqSubDirName)]
Seqname=ak.unflatten(Seqname, counts)
ila['Isolates']=ak.with_field(ila.Isolates, Seqname, where="Seqname")

In [22]:
# process same-strain into json and then awkwardarray
#sd = (same_strain_list.groupby(['RandomID']).aggregate(list)).to_dict()
from presnakemake_processing import process_same_strain_file
same_strain_list=process_same_strain_file("~/PycharmProjects/mge-project/workflow/config/All_same_strain_pairs.xlsx")
ssj = (same_strain_list.groupby('RandomID')['Seqplates']
             .apply(list)
             .reset_index()
             .rename(columns={0:'Isolates'})
             .to_json(orient='records'))

#print(json.dumps(json.loads(ssj), indent=2, sort_keys=True))
ssa=ak.from_json(ssj)

In [23]:
# merge two arrays in outer join
dictionary, index = np.unique(np.asarray(ssa.RandomID), return_index=True)
closest = np.searchsorted(dictionary, np.asarray(ila.RandomID), side="left")
is_within_range = ak.Array(closest).mask[closest < len(dictionary)]
is_good_match = ak.Array(dictionary)[is_within_range] == ila.RandomID
print(ila.RandomID)
reordering = ak.Array(closest).mask[is_good_match]
ila["Seqplates"] = ssa.Seqplates[index][reordering]


[0, 14199, 20578, 20643, 23595, ... 3247835, 3248787, 3251316, 3257723, 3267020]


In [24]:
# Add same-strain using cartesian product
cart=ak.cartesian([ila.Isolates.Seqname, ila.Seqplates], nested=True)
is_ss=ak.any(cart.slot0==cart.slot1, axis=-1)
is_ss=ak.fill_none(is_ss, False)
ila['Isolates']=ak.with_field(ila.Isolates, is_ss, where="IsSameStrain")

## Task #2: import pangenome coverage data
The absolute necessaries:
 - gene names (from roary)
 - gene functions, scaffolds (from prokka)
 - gene coverages (from coverage)
 - BRESEQ
 
All else can wait!
### Work plan
 Load into pangenomes structs and then merge them with dataset at very end

In [285]:
# get all gene_presence_absence files (older version, works)


# import pyarrow as pa
# from pyarrow import csv
# import pyarrow.parquet as pq

# datadir=os.getcwd()+"/PycharmProjects/mge-project/data/"

# dirnames=np.char.add(np.asarray(datadir+"/pangenome/", dtype=str), np.asarray(ila.NewRandomID,dtype=str))
# filenames=np.char.add(dirnames,np.asarray('/gene_presence_absence.csv', dtype=str))

# isfile=[os.path.isfile(fn) for fn in filenames]

# def ak_string_to_int(akstr):
#     nparr=ak.to_numpy(akstr,allow_missing=True)
#     nparr=np.fromstring(nparr, dtype=int)
#     akint=ak.Array(nparr)
#     return akint


# #get pandas table
# def get_pangenome_data(filename,opt):
#     roary_pa=csv.read_csv(filename, convert_options=opt)
#     roary_pa=roary_pa.rename_columns([n.replace(' ', '_').replace('.','') for n in roary_pa.column_names])
#     roary_pa=roary_pa.drop([col for col in roary_pa.column_names if 'Sample' in col])
#     print(filename)
#     roary_ak=ak.from_arrow(roary_pa)
#     # roary_ak.Accessory_Fragment=ak_string_to_int(roary_ak.Accessory_Fragment)
#     # roary_ak.Accessory_Order_with_Fragment=ak_string_to_int(roary_ak.Accessory_Order_with_Fragment)

#     return roary_ak, ak.num(roary_ak.Gene, axis=0)




# ilp=ila[:4]
# isfile=isfile[:4]
# filenames=filenames[:4]

# opt=csv.ConvertOptions(column_types=[('Accessory Fragment', pa.string()),( 'Accessory Order with Fragment', pa.string() )],
#         strings_can_be_null=True)



# rda=ak.to_arrow(ilp)

# roary_data=[ (get_pangenome_data(fn, opt))  for fn in filenames[isfile] ]

# r_count=np.array([c[1] for c in roary_data])
# count=np.zeros((len(ilp),),dtype=int)
# count[isfile]=r_count

# roary_data=[c[0] for c in roary_data]


# rd=ak.concatenate(roary_data,merge=False, mergebool=False)
# rd=ak.unflatten(rd,count)

# ilp=ak.with_field(ilp, rd, where='Pangenome')
# print(ak.type(ilp))

# ak.to_parquet(ilp, datadir+"/datastruct/up_to_pangenome.parquet")


/media/kishonylab/KishonyStorage/noga/PycharmProjects/mge-project/data//pangenome/20643/gene_presence_absence.csv
4 * {"NewRandomID": int64, "RandomID": int64, "CaseNum": int64, "Seqplates": option[var * string], "Isolates": var * {"IsoNum": float64, "RowNum": int64, "SampleDate": string, "SeqSubDirName": string, "LowQualityThrs": int64, "Freezer_pos": string, "Used": int64, "SiteString": string, "Seqname": string, "IsSameStrain": bool}, "Pangenome": var * {"Gene": option[string], "Non-unique_Gene_name": option[string], "Annotation": option[string], "No_isolates": ?int64, "No_sequences": ?int64, "Avg_sequences_per_isolate": ?int64, "Genome_Fragment": ?int64, "Order_within_Fragment": ?int64, "Accessory_Fragment": option[string], "Accessory_Order_with_Fragment": option[string], "QC": option[string], "Min_group_size_nuc": ?int64, "Max_group_size_nuc": ?int64, "Avg_group_size_nuc": ?int64}}


ArrowInvalid: Length spanned by list offsets (5305) larger than values array (length 5304)

In [296]:
# get all gene_presence_absence files (older version, works)


import pyarrow as pa
from pyarrow import csv
import pyarrow.parquet as pq

datadir=os.getcwd()+"/PycharmProjects/mge-project/data/"

dirnames=np.char.add(np.asarray(datadir+"/pangenome/", dtype=str), np.asarray(ila.NewRandomID,dtype=str))
filenames=np.char.add(dirnames,np.asarray('/gene_presence_absence.csv', dtype=str))

isfile=[os.path.isfile(fn) for fn in filenames]

def ak_string_to_int(akstr):
    nparr=ak.to_numpy(akstr,allow_missing=True)
    nparr=np.fromstring(nparr, dtype=int)
    akint=ak.Array(nparr)
    return akint

ilp=ila

roary_pd= [pd.read_csv(fn, error_bad_lines=False) for fn in filenames[isfile]]

count=np.zeros((len(ilp),),dtype=int)
count_r= [len(rpd.index) for rpd in roary_pd]
count[isfile]=count_r

merged_rpd=pd.concat(roary_pd, ignore_index=True)
par=pa.Table.from_pandas(merged_rpd)
rd=ak.from_arrow(par)

rd=ak.unflatten(rd,count)

ilp=ak.with_field(ilp, rd, where='Pangenome')

print(ak.type(ilp))

ak.to_parquet(ilp, datadir+"/datastruct/up_to_pangenome.parquet")

_SeqPlate15_D6": option[string], "Sample_Maccabi_Ecoli_SeqPlate22_D8": option[string], "Sample_Maccabi_Ecoli_SeqPlate25_F9": option[string], "Sample_Maccabi_Ecoli_SeqPlate11_D1": option[string], "Sample_Maccabi_Ecoli_SeqPlate25_C6": option[string], "Sample_Maccabi_Ecoli_SeqPlate8_D3": option[string], "Sample_Maccabi_Ecoli_SeqPlate8_E3": option[string], "Sample_Maccabi_Ecoli_SeqPlate5_B6": option[string], "Sample_Maccabi_Ecoli_SeqPlate5_B7": option[string], "Sample_Maccabi_Ecoli_SeqPlate5_B8": option[string], "Sample_Maccabi_Ecoli_SeqPlate10_E5": option[string], "Sample_Maccabi_Ecoli_SeqPlate10_F5": option[string], "Sample_Maccabi_Ecoli_SeqPlate13_F2": option[string], "Sample_Maccabi_Ecoli_SeqPlate17_D11": option[string], "Sample_Maccabi_Ecoli_SeqPlate19_G2": option[string], "Sample_Maccabi_Ecoli_SeqPlate22_F1": option[string], "Sample_Maccabi_Ecoli_SeqPlate20_B1": option[string], "Sample_Maccabi_Ecoli_SeqPlate22_B12": option[string], "Sample_Maccabi_Ecoli_SeqPlate20_H11": option[string

ArrowNotImplementedError: Lists with non-zero length null components are not supported